# Lesson 3: Agentic Search

In [11]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [12]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


"The Blackwell GPU features advanced architecture for AI, with up to 192GB of HBM3e memory and 1.4 exaFLOPS of compute power. It uses TSMC's 4nm process for high performance. Blackwell is designed for generative AI and accelerated computing."

## Regular search

In [13]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [14]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [15]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [16]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://weather.com/weather/today/l/USCA0987:1:US


<body class="__className_f367f3 font-sans"><div hidden=""><!--$--><!--/$--></div><script>(self.__next_s=self.__next_s||[]).push(["/wx-next-web/static/vendor/@mparticle/web-sdk@2.47.1/dist/mparticle.min.js",{}])</script><div class="flex items-start"><div aria-hidden="false" class="sticky top-0 z-1 hidden md:block"><nav aria-label="Site Navigation Links" class="top-0 box-content flex h-screen w-[72px] flex-col overflow-hidden border-r-1 border-[#2524221A] bg-white leading-none transition-transform duration-300 xl:w-[260px]" data-testid="sidebar-navigation"><div class="mb-0 flex items-center p-3 leading-none"><button class="block h-12 w-12 rounded-lg border-none transition-colors duration-200 hover:bg-gray-100 focus:bg-gray-100 focus:outline-none xl:mr-3 xl:hidden openSidebarOverlayButton" tabindex="0" type="button"><svg class="inline-block w-6 h-6 text-gray-900" fill="no

In [17]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/USCA0987:1:US


Home Home Today Today Hourly Hourly 10 Day 10 Day Monthly Monthly Radar Radar Video Video Explore Explore Home Home Today Today Hourly Hourly 10 Day 10 Day Monthly Monthly Radar Radar Video Video Explore Explore Wind Humidity Air Quality Dew Point Pressure UV Index Visibility Moon Phase Sunrise Sunset Hourly Weather Don't Miss A Rare Southern Snowstorm: What You Need To Know Multiple Systems To Bring Snow To Great Lakes, Northeast Massive Pileup Along I-81 With Wintry Conditions What To Do If You Find A Cold-Stunned Iguana 0:31 A Rare Southern Snowstorm: What You Need To Know Multiple Systems To Bring Snow To Great Lakes, Northeast Massive Pileup Along I-81 With Wintry Conditions What To Do If You Find A Cold-Stunned Iguana 0:31 Seasonal Hub Thousands Gather At Stonehenge For Winter Solstice 0:24 The Top 5 Strangest White Christmases In Recent Memory 0:47 Do You Really Lose Most Of Your Body Heat Through Your Head? 0:35 Frost

## Agentic Search

In [18]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1768667427, 'localtime': '2026-01-17 08:30'}, 'current': {'last_updated_epoch': 1768666500, 'last_updated': '2026-01-17 08:15', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 38, 'wind_dir': 'NE', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 75, 'feelslike_c': 6.4, 'feelslike_f': 43.5, 'windchill_c': 9.7, 'windchill_f': 49.5, 'heatindex_c': 11.1, 'heatindex_f': 52.0, 'dewpoint_c': 7.1, 'dewpoint_f': 44.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.2, 'gust_mph': 14.0, 'gust_kph': 22.5}}


In [19]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1768667427,
        "localtime": "2026-01-17 08:30"
    },
    "current": {
        "last_updated_epoch": 1768666500,
        "last_updated": "2026-01-17 08:15",
        "temp_c": 8.3,
        "temp_f": 46.9,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 6.9,
        "wind_kph": 11.2,
        "wind_degree": 38,
        "wind_dir": "NE",
        "pressure_mb": 1021.0,
        "pressure_in": 30.15,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 93,
        "cloud": 75,
        "feelslike_c": 6.4,
        "feelslike_f": 43.5,
        "windchill_c": 9.7,
        "

<img src="./google_sample.png" width="800" height="600">